# Project 1: Assembling Genes

   <div class="alert alert-block alert-danger">
    <center>Due: <b>Monday, September 5, 8:59pm</b>.</center> 
   </div>
   
   <div class="alert alert-block alert-warning">
   <center>
       <b>Collaboration and Resource Policy</b>
    </center>
    
For this assignment, you are encouraged to work with one other person satisfying the constraints from Class 2. 
You are permitted (actually _encouraged_) to discuss these problems with anyone you want, including other students in the class. If you do discuss the specific questions in the assignment with anyone other than your assignment partner and the course staff, though, you should list them in the _External resources used_ section below.
    
You are welcome to use any resources you want for this assignment, other than ones that would defeat the purpose of the assignment. This means you should not look at answers or code from previous semesters of this course, or from any other students in the class (other than your collaboration with your partner), and if you find code that implements the problem you are being asked to do for the assignment, you should not use that code. You should document all external resource you use that are not part of the course materials in the _External resources used_ section below.
    </div>

**Team submitting this assignment:**  
<div class="alert alert-success">
    <b><em>list each member of your team here, including both your name and UVA computing id</em></b>

Team Members (Names): Esha Tulsian and Amelia Norman 

Team Member UVA Computing IDs: et4xam and aln4t

</div>

**External resources used:** 
<div class="alert alert-success">
<em>It is not necessary to list the course materials, but if you used any other resources, including discussing problems with students not on your team, list them here.</em>
    
External Resources Used:

Classmates: Wamia Said
    
(For further clarification of Eulerian paths): https://cp-algorithms.com/graph/euler_path.html#algorithm
    
(For further clarification of DFS): https://www.programiz.com/dsa/graph-dfs
</div> 

In this project, we will explore genome assembly—the process of determining the order of nucleotides in DNA from fragmented reads. As you might have studied in the reading assignments, genome assembly can get quite complicated, as problems such as full sequence coverage, finding a good length for reads (the $k$ in $k$-mer), and sequencing errors present challenges for sequencing analysis and accuracy. You can assume perfect coverage for all parts of the assignment and no read errors for the first two questions.


<b>Submission</b>: Please submit the code you wrote to generate your answers for all parts using this form: <a href="https://forms.gle/rNTXfYojTLEQ8idg6"><em>https://forms.gle/rNTXfYojTLEQ8idg6</em></a>. Your answers should be in the Jupyter Notebook, along with your code. Before submission, you should make a copy of your notebook file with the name <i>uvaid1\_uvaid2.ipynb</i> (where <i>uvaidn</i> is each teammates UVA id) so the submitted file identifies you. You and your partner should submit a single file once together. Submission is due 8:59 pm (EST) on Monday, September 5.

## Install basic required packages.

- Install basic required packages, should be run only once. You may need to restart the kernel after this stage.
- Make sure you have [graphviz](https://graphviz.org/download/) installed on your system.
- The second cell adds Graphviz to your path, you may have to change based on where the install folder is.

<b>NOTE: We provide utils.py, which may contain helpful functions for you to use, as well as gvmagic.py, which is a deprecated package to use graphviz within the notebook</b>

In [1]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
os.environ["PATH"] += os.pathsep + 'C:/Program Files/Graphviz/bin'

## Genome Assembly

For this part, you're given reads generated while trying to sequence the DNA of a TeleTubby (some unknown organism) with a \textit{very} small genetic code. By answering the following questions, you will learn how to assemble the original genome sequence from sequence reads.

Sequencing data is often stored in FASTQ file format. In TeleTubby.fastq, you will find the data organized in a particular order that repeats every four lines. The first line contains the metadata that encodes the name of the read, the experiment type, the kind of sequencing machine used, etc. The second line is the sequence of bases. The third line functions as a placeholder line. The fourth line is a sequence of base qualities that encode the qualities for the corresponding bases in the sequence line. We will only work with the sequence and quality score lines in this question.

In [3]:
import collections
import matplotlib.pyplot as plt
import numpy as np
import utils
from tqdm import tqdm
import pandas as pd
import statistics

#### Question 1.1.1 GC-content

The GC-content (or the ratio of G and C nucleotides) is related to the melting temperature of the DNA double helix. Use the following equation to calculate the melting temperature of DNA for TeleTubby $t_m$ in Celsius:

\begin{equation*}
t_m = 64.9+0.41(\%GC)-\frac{500}{\text{length of sequence}}
\end{equation*}

As a reference, the human genome is known to have between 35%-60% GC-content. 

In [4]:
# Read sequence reads (error-free) from file
sequence_reads, qualities = utils.read_fastq('TeleTubby.fastq')
tot = len(sequence_reads)

# Find total pairs
sequences = tot * 8
# Find individual pair values
vals = []
for i in range(tot):   # adds chars from genome to vals
    temp = list(sequence_reads[i])
    for j in temp:
        vals.append(j)

In [5]:
# Find # of C--G pairs
count = 0
for i in range(sequences):
    if vals[i] == 'C' or vals[i] == 'G':
        count += 1

In [6]:
# Calculate %GC content
GC_pct = (count/sequences)   # ASK: should this be a decimal or whole number?
print(GC_pct)

0.47952218430034127


In [7]:
# Print out temperature in Celsius
temp_tt = 64.9 + (0.41*GC_pct) - (500/sequences)
print(temp_tt)
# redo this

64.88329351535837


#### Question 1.1.2 Interpreting quality scores

Phred33 quality scores are represented as the character with an ASCII code equal to its value + 33 (to make them easy to print alongside genome sequences). List the top 5 most frequent scores in ASCII symbol as well as their Phredd33 scores in TeleTubby.fastq. You can refer to the [official Illumina website](https://support.illumina.com/help/BaseSpace_OLH_009008/Content/Source/Informatics/BS/QualityScoreEncoding_swBS.htm) to reference the scoring encoding.

What is the average Phred33 score in TeleTubby.fastq?

In [8]:
# separate vals
scores = []
for i in range(tot):   # adds chars from genome to vals
    temp = list(qualities[i])
    for j in temp:
        scores.append(j)
        
scores_df = pd.DataFrame({"scores":scores})   # makes it easier to use functions

top_5 = scores_df["scores"].value_counts().head(5).index.values   # automatically sorted in descending order
top_5 = [top_5[i][0] for i in range(5)]

In [9]:
# calculate Phred33 score
phred33 = []
# convert ASCII+33
for i in top_5:
    phred33.append(ord(i)-33)

In [10]:
# Create DF; print ascii and phred33 scores
freq_qualities = pd.DataFrame({"ASCII":top_5, "Phred33":phred33})
freq_qualities

,ASCII,Phred33
0,5,20
1,D,35
2,?,30
3,K,42
4,F,37


In [11]:
# Calculate average Phred33 scores
all_phred33 = []
for i in scores:
    all_phred33.append(ord(i)-33)
mean_phred33 = statistics.mean(all_phred33)
mean_phred33

34.476535836177476

#### Question 1.1.3 Frequency analysis

Looking at repetitions in the sequence can be helpful in estimating the "redudancy" in the organisms. Humand and other evolved animals have a lot of redundancy, while smaller organisms like bacteria have highly packed genomes. One heuristic to estimate this before actually performing the assembly could be looking at how often certain $k$-mers are repeated.

<b>Print out the 3 most frequent k-mers with their frequencies</b>

In [12]:
# Find and print out the three most repeated k-mers and their frequencies
def calc_kmers(reads):
    sequence_lst = []   # temp storage for finding most common seq
    for seq in reads:
        for pos in range(8):   # grabbing individual characters
            end_pt = 7-pos   # distance to end of string
            temp = 1   # determines length of sub-sequence 
            for i in range(end_pt):
                subset = seq[pos:(pos+temp+1)]
                sequence_lst.append(subset)
                temp += 1
    return sequence_lst   # for later calculation of full genome

In [13]:
# develop list of full genome
kmers = calc_kmers(sequence_reads)
#print(len(kmers))

# check if we have all values calculated (later)
#print(28*293)   # num vals we should have

In [14]:
# pandas again, for the easy peasy value_counts()
all_kmers = pd.DataFrame({"kmers":kmers})
top_3 = all_kmers["kmers"].value_counts().head(3)
top_3_kmers = top_3.index.values
top_3_freq = list(top_3)
top_kmers = pd.DataFrame({"k-mers": top_3_kmers, "Frequency": top_3_freq})
top_kmers

,k-mers,Frequency
0,GC,165
1,TG,157
2,CA,152


### Question 1.2. Greedy approach

One of the approaches to assemble the genome from the given reads is a greedy algorithm. Have a look at the greedy algorithm described on [Wikipedia](https://en.wikipedia.org/wiki/Sequence_assembly#Greedy_algorithm) and answer the following.

#### Question 1.2.1 What would the runtime be of this algorithm, given $n$ $k$-mer reads?

<i>Answer</i>: O(k*n^2)

#### Question 1.2.2 Would this algorithm always yield a unique solution?

<i>Answer</i>: No, the greedy algorithm looks for the optimal solution at any given moment, but that does not always result in unique solutions.

#### Question 1.2.3 Would this algorithm always yield the <i>right</i> solution?

<i>Answer</i>: No because this algorithm makes decisions based on the local information present at the moment without taking into account the overall or global problem, so it does not always result in the global right solution but does choose the local best solution.

### Question 1.3 Graph-based approaches

Graphs for genome assembly can be constructed in two ways:

- de Bruijn graph: Processing $(k-1)$mers as nodes, with $k-$mers as edges, and
- Overlap graph: Processing $k-$mers as edges, with $(k-1)-$mers as nodes.

de Bruijn graphs can be processed to find Euler paths, while Overlap graphs can be processed to find Hamiltonian paths. Both of these are valid ways to reconstruct the original genome.

<b>Use one of these two techniques to reconstruct the sequence, and print out your reconstructed sequence. Which method did you pick out of the two, and why? (hint: imagine what would happen when we have millions of reads). Use the k-mers provided in TeleTubby.fastq</b>.

We provide some skeleton code that you may use, but you may also come up with your own solution.

**Textual Response:**

We selected the de Bruijn graph because it was easier to mentally visualize the paths between $(k-1)$-mers as being the the full $k$-mer (thereby having the last character being a direct path to the next $(k-1)$-mer) than the other way around (where the edge is made up of a combination of the nodes it lies between).  

In [36]:
# Read reads into graph
# using de Bruijn graph; 
# prefix = first 7 (k-1) chars
# suffix = last 7 (k-1) chars
# if suffix(A) == prefix(B), edge A-->B 

def build_graph(k_mers):
    edges = []
    nodes = set()
    
    # Your code here
    for i in k_mers:
        # add prefix, suffix nodes (k-1)-mers
        nodes.add(i[1:])
        nodes.add(i[:-1])
        
        # edges = kmers
        edges.append(i)
    
    return nodes, edges   # skeleton code says this is okay; o/w, make a graph

nodes, edges = build_graph(sequence_reads)

In [37]:
# find Eulerian path
import copy
import random
        
def assemble_sequence(n, e, k=8):
    # calculate start node
    incoming_nodes = []
    outgoing_nodes = []
    for edges in e:
        incoming_nodes.append(edges[:-1])
        outgoing_nodes.append(edges[1:])
    
    start_node = ""
    for i in incoming_nodes:
        if i not in outgoing_nodes:
            start_node = i
    
    # find eulerian path
    # code here adapted from this pseudocode for euler: https://cp-algorithms.com/graph/euler_path.html#algorithm
    # and this pseudocode for DFS: https://www.programiz.com/dsa/graph-dfs
    stack = [start_node]
    path = []
        
    while len(stack) != 0:
        # end condition is n is of degree 0 / if there are no more outgoing nodes
        end_edge = True
        # following implements a DFS; goes to end of sequence as far as possible then builds up
        for edge in e:
            if edge[:(k-1)] == stack[-1]:
                e.remove(edge)   # edge traversed
                stack.append(edge[1:])   # check following node
                end_edge = False   # means that this is not the end; continue
                break
        if end_edge:
            path.insert(0, stack.pop(-1))   # append stack value to path
    
    # assemble actual sequence
    final_sequence = start_node[:(k-2)]   # add first few chars from start node; start node not in eulerian path
    for node in range(len(path)):
        final_sequence += path[node][-1]   # should only differ by 1 char at end; append this value
    
    return final_sequence

In [38]:
assembled_sequence = assemble_sequence(nodes, edges)
print(len(assembled_sequence))
assembled_sequence

300


'TACGCCAAATAGCAATGCGCAGGATAACAACTTATGTACTACATGTTGTTTCTCGTGCCCGCCAATGTCGAGAGATTTGTGCTATCGCAACCTAAGAGAGAAGGGGTTTTGTGTTAGCAGTTTCTTCATGCATCTCTTTACAAGAATTACAGGAGCCAAACACTCGCTGTCATGGTATCGACATATCGCTGCCCGGAGGCGCTATCGCAAACCGACTGTCGGACTCTTTCATGAGCAAAAAAAGTGGGAGTATGGTGCACATCCGCTATCGCTACTGGTGCCGCCCTTCGATGCAATGTT'

In [39]:
for n in sequence_reads:
    if n not in assembled_sequence:
        print(n)   # GENOME SEQUENCED!

Personal notes working through issues:

Recursion through a dictionary that contained values of just outgoing k-mers or both incoming and outgoing k-mers in a list of lists. I also attempted to create a Eulerian cycle, which was successful (sometimes, if starting from a node other than the start node). The issue here is that the recursion traverses the end node before it's supposed to and removes it from the list, thereby ruining the order of nucleotides and resulting in more characters in the final solution. I was really struggling to fix this problem and deleted my notebook to start over with another method.

Took a break, consulted https://cp-algorithms.com/graph/euler_path.html#algorithm, worked on DFS in AI and decided to integrate DFS and remove the conditional problem (the end node) by just starting there and working up.

## Question 2 - Sequencing SARS-CoV-2 virus

Let's move on from TeleTubbies to real-world organisms. Let's start small- with a variant of the SARS-CoV-2 virus. You're given reads from <i>actual</i> genome sequencing runs in the SARS-CoV2.fastq file provided.

Repeat Question 1.3 on this data. You can re-use your implementation and simply run it on the new data. Print out your reconstructed sequence to a file "output.txt". For this part, we will still assume that all the reads are error-free. Set $k=25$.

In [19]:
import sys
sys.setrecursionlimit(2000)

In [20]:
# Read sequence reads
sequence_reads_covid, qualities_covid = utils.read_fastq('SARS-CoV2.fastq')

In [21]:
# Read reads into graph
cg_nodes, cg_edges = build_graph(sequence_reads_covid)

In [22]:
# assemble sequence
covid_path = assemble_sequence(cg_nodes, cg_edges, 25)

In [23]:
len(covid_path)

29903

In [24]:
# check assembly
for n in sequence_reads_covid:
    if n not in covid_path:
        print(n)

In [25]:
# Write assembled sequence to file

assmebled_seq = covid_path # Use your assembled genome
with open("covid_overlap.txt", "w") as f:
    f.write(assmebled_seq)

# Question 3- Error-Aware Assembly (Extra Credit)

In the parts above, we assumed error-free reads while assembling $k$-mers. As much as we'd like that, actual reads can (and do) have errors, captured by their Phred scores. For this question, you're given raw, actual reads from sequencing runs (download reads here: https://sra-pub-sars-cov2.s3.amazonaws.com/sra-src/SRR11528307/ABS2-LN-R1_cleaned_paired.fastq.gz).  Given these reads and their Phred33 scores, can you assemble the genome?

<b>Print out your assembled sequence, along with a brief explanation of how your algorithm works</b>

This is an open-ended question. You are free to use any approach to deal with the issue. Make sure you provide your code, along with any assumptions you may have, in the cells below.

In [ ]:
# Issues: identify bubbles, distinguish bubbles from repeated/slightly differing reads that aren't erroneous